In [1]:
# Aurthor: Elihu Essien-Thomspon
# Program Description: Experiment 2
# Finding the best Ant Colony Optimization
# Algorithm Representative to use for
# the final Experiment.

import numpy as np
import random
import math

In [2]:
# Returns the distance between two 
# points(x, y) on the map

def cityDistance(map, cityID1, cityID2):
    x = abs(map[cityID1][0] - map[cityID2][0])
    y = abs(map[cityID1][1] - map[cityID2][1])
    
    # pythagoras
    dist = math.sqrt(x**2 + y**2)
    return(dist)

In [3]:
# Reading in map data from text file
# and returning a distance matrix to use
# for calculations
def getMap(size,id):
    cities = np.empty((0,2), dtype=int)      # initialize empty 2D vector list
    
    # open file
    with open(f"C:/Users/C14460702/Dissertation/Data/Maps/Size - {size}/map{id}.txt", 'r') as f:
        # extract vector list data from the comma separated string list data
        for line in f:
            line = line.strip('\n')    # remove '\n'
            vec = line.split(', ')     # split by ', '
            
            # cast co-ordinate vector as integers and insert into list 
            cities = np.append(cities, [[ int(vec[0]), int(vec[1]) ]], axis=0)
    
    
    # initialize distance table
    mapSize = len(cities)
    distanceTable = np.zeros((mapSize,mapSize))
    for row in range(mapSize):
        for col in range(mapSize):
            distanceTable[row][col] = cityDistance(cities, row, col)
    
    # return distance matrix
    return (distanceTable)

In [4]:
# Function takes in a route list
# and a distnce matrix to be used
# to score the route given

def routeScore(route, distanceTable):
    
    dist = 0;
    
    # aggregate distances between cities
    # on the list given
    for i in range(len(route)):
        cur = route[i]
        nxt = route[(i+1) % len(route)]  # includes loop back to start
        dist += distanceTable[cur][nxt]  # get the distances from the matrix
        
    
    # the smaller the distance, the higher the score
    return (1/dist)

In [5]:
class Particle:
    def __init__(self, route, distanceTable):
        self.route = route
        self.distanceTable = distanceTable
        self.score = routeScore(route, distanceTable)
        self.pBest = [self.route, self.score]
        self.velocity = []  #swap sequinces to transform map state
    
    def updateRecords(self):
        self.score = routeScore(self.route, self.distanceTable)
        if(self.score > self.pBest[1]):
            self.pBest = [self.route, self.score]
            
    def getRoute(self):
        return(self.route)
    
    def getScore(self):
        return(self.score)
    
    def caculateNewVelocity(self, bestSolutionFound):
        # stochastic probabilities for adding swap sequences
        # to the final velocity
        pBestProb = random.random()
        gBestProb = random.random()
        
        # calculate swap sequence to get to pBest
        cur = [i for i in self.route] # temporary list used to validate swap sequence logic
        nxt = [i for i in self.pBest[0]] # target route
        for i in range(len(cur)):
            if(cur[i] != nxt[i]):
                j = cur.index(nxt[i])            # position to swap with
                cur[i], cur[j] = cur[j], cur[i]  # swap temp list to keep track
                
                # add current swap to velocity list using pBest probability
                if (random.random() < pBestProb):
                    self.velocity += [[i, j]]
        
        # calculate swap sequence to get to gBest
        cur = [i for i in self.route] # temporary list used to validate swap sequence logic
        nxt = [i for i in bestSolutionFound] # target route
        for i in range(len(cur)):
            if(cur[i] != nxt[i]):
                j = cur.index(nxt[i])            # position to swap with
                cur[i], cur[j] = cur[j], cur[i]  # swap temp list to keep track
                
                # add current swap to velocity list using gBest probability
                if (random.random() < gBestProb):
                    self.velocity += [[i, j]]
        
    def move(self):
        # swap cities using swap sequence
        for swap in self.velocity:
            self.route[swap[0]], self.route[swap[1]] = self.route[swap[1]], self.route[swap[0]]
        
        # update new score
        self.score = routeScore(self.route, self.distanceTable)
        
        # clear velocity
        self.velocity = []

In [9]:
def PSO(mapSize, popSize, maxIterations, distanceTable, alpha=None, beta=None, itPerc = None):
    # generate cityIDs list representing the 
    # default chromosome or individual
    defaultRoute = [i for i in range(mapSize)]
    
    # Step 1 - Create initial population
    # populate list with initial random individuals
    particles = [Particle(random.sample(defaultRoute, mapSize), distanceTable) for i in range(popSize)]
        
    # records
    GBestScore_PerItreation = []
    gBest = [[],0] # [route, score]
    tuningProb = alpha
        
    for iteration in range(maxIterations):
        # Step 2 - Cacluate and Update Record
        
        # get the best solution found in this iteration
        gcBest = [[],0]  # [route, score]
        for member in particles:
            if(member.getScore() > gcBest[1]):
                gcBest = [member.getRoute(), member.getScore()]
        
        # get the all time best solution found
        if(gcBest[1] > gBest[1]):
                gBest = [i for i in gcBest]
        
        GBestScore_PerItreation += [gBest[1]]
        
        
        # Step3 - calculate velocities and move particles
        if(alpha==None): # original TSP-PSO algorithm
            for member in particles:
                member.caculateNewVelocity(gBest[0])
                member.move()
        else: # new TSP-MPSO algorithm
            if(random.random() < tuningProb): # use the gbest
                for member in particles:
                    member.caculateNewVelocity(gBest[0])
                    member.move()
            else:                             # use the gcbest
                for member in particles:
                    member.caculateNewVelocity(gcBest[0])
                    member.move()
            if(tuningProb <= beta):           # move from alpha% to beta% over the first [itPerc] of iterations
                tuningProb += (alpha-beta)/(popSize * itPerc)
    
    return(GBestScore_PerItreation)

In [7]:
# Experiment 3: testing the PSO vs MPSO
# the MPSO algorithm is run twice
# once with an iteration percetage rate of 1/4
# and the second with 1/2
popSize = 50            # Number of chromosomes to use
maxIterations = 100    # Maximum number of generations
alpha = 10
beta = 90

# Traveling Salesman Problem Settings
mapSize = 10        # cities per map
mapCounter = 0


f1 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment3/Size - {mapSize}/PSO.txt", 'w')
f2 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment3/Size - {mapSize}/MPSO(50%).txt", 'w')
f3 = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment3/Size - {mapSize}/MPSO(25%).txt", 'w')

while (True):
    try:
        # get map
        distanceTable = getMap(mapSize,mapCounter)
        
        
        data1 = PSO(mapSize, popSize, maxIterations, distanceTable)
        data2 = PSO(mapSize, popSize, maxIterations, distanceTable, alpha, beta, 0.5)
        data3 = PSO(mapSize, popSize, maxIterations, distanceTable, alpha, beta, 0.25)
        f1.writelines(str(data1))
        f2.writelines(str(data2))
        f3.writelines(str(data3))
        
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break

f1.close()
f2.close()
f3.close()

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish